In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from Network import Network
from GridSearch import GridSearcher
from activation_functions import sigmoid, relu,relu_derivative, sigmoid_derivative, tanh, tanh_derivative
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [2]:
# Set random seed for reproducibility
np.random.seed(42)
df_vote = pd.read_csv("CongressionalVotingID.shuf.lrn.csv", index_col="ID")
df_vote = df_vote.applymap(lambda x: 1 if x == "y" else 0 if x == "n" else x)
df_vote = df_vote.replace("democrat", 1)
df_vote = df_vote.replace("republican", 0)
df_vote = df_vote.replace("unknown", 0)

In [3]:
x_train, x_test, y_train, y_test = train_test_split(df_vote.iloc[:,1:], df_vote.iloc[:, 0],test_size=0.4, random_state=11)

In [10]:
# x_train = np.expand_dims(x_train.to_numpy(), axis=1)
# y_train = np.expand_dims(y_train.to_numpy(), axis=1)
# x_test = np.expand_dims(x_test.to_numpy(), axis=1)
# y_test = np.expand_dims(y_test.to_numpy(), axis=1)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)
learning_rate=0.1

In [5]:
network = Network(learning_rate, epochs=1000, node_counts = [3, 3, 3],activation_function=(tanh, tanh_derivative))

In [6]:
params = {
    "learning_rate": [0.2],
    "epochs": [10],
    "node_counts": [  [3], [3, 3]],
    "activation_function":[(sigmoid,sigmoid_derivative)]
}
gs = GridSearcher(network, params, X_train, y_train)
gs_results = gs.search()
gs_results

1 / 2
2 / 2


,params,test_score,fit_time,score_time,score_mean
0,"{'learning_rate': 0.2, 'epochs': 10, 'node_cou...","[nan, nan, nan, nan, nan]","[0.17290019989013672, 0.16590356826782227, 0.1...","[0.0049974918365478516, 0.0049974918365478516,...",NaN
1,"{'learning_rate': 0.2, 'epochs': 10, 'node_cou...","[nan, nan, nan, nan, nan]","[0.33780670166015625, 0.3747837543487549, 0.46...","[0.0029990673065185547, 0.004999399185180664, ...",NaN


In [7]:
# params=gs_results.iloc[gs_results['score_mean'].idxmax()]
# comment/delete the below line when grid search works with valid output
params=gs_results.iloc[0] 

selected_prams=params["params"]
selected_prams

{'learning_rate': 0.2,
 'epochs': 10,
 'node_counts': [3],
 'activation_function': (<function activation_functions.sigmoid(x)>,
  <function activation_functions.sigmoid_derivative(x)>)}

In [9]:
new_network= Network(learning_rate=selected_prams["learning_rate"], epochs=selected_prams["epochs"], node_counts=selected_prams["node_counts"],activation_function=selected_prams["activation_function"]
)

In [10]:
new_network.fit(x_train, y_train)

out = new_network.predict(x_test)

TypeError: type numpy.ndarray doesn't define __round__ method

In [ ]:
y_pred=new_network.predict(x_test)
y_pred=np.where(np.array(y_pred) > 0.5, 1, 0)


In [ ]:
def score(y_true, y_pred):
    correct_predictions = 0
    total_predictions = len(y_true)

    for true_label, predicted_label in zip(y_true, y_pred):
        if true_label == predicted_label:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy
accuracy=score(y_test,y_pred)
print("accuracy")
print(accuracy)

***Classification report for Self Neural Network***

In [ ]:
print(classification_report(y_pred=np.squeeze(y_pred),y_true=np.squeeze(y_test)))

***Tensor Flow***

In [281]:
def train_model(x_train,y_train, num_nodes, dropout_prob,lr, batch_size, epochs):
    nn_model=tf.keras.Sequential([tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(16,)),
                              tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(num_nodes, activation='relu'),
                             tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="binary_crossentropy",metrics=['accuracy'])

    #x_train=x_train.astype(float)

    history = nn_model.fit(x_train,y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    return nn_model, history

In [282]:
nn_model, history= train_model(np.squeeze(x_train),np.squeeze(y_train),16 , 0,0.1, 32, 1000)

In [ ]:
y_pred=nn_model.predict(np.squeeze(x_test))
y_pred=(y_pred>0.5).astype(int).reshape(-1)
y_pred


print(classification_report(y_pred,np.squeeze(y_test)))